In [2]:
import keras

In [4]:
from keras.models import Sequential
obj = Sequential()

In [6]:
import pandas as pd
dt = pd.read_csv('/content/kidney_disease.csv')
dt.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [8]:
dt.corr()

ValueError: could not convert string to float: 'normal'

In [12]:
# Select only numeric columns for correlation calculation
numeric_dt = dt.select_dtypes(include=['number'])

# Calculate the correlation on the numeric data
correlation_matrix = numeric_dt.corr()

# You can then display or use the correlation_matrix as needed
print(correlation_matrix)

            id       age        bp        sg        al        su       bgr  \
id    1.000000 -0.185308 -0.245744  0.642156 -0.541993 -0.283416 -0.338673   
age  -0.185308  1.000000  0.159480 -0.191096  0.122091  0.220866  0.244992   
bp   -0.245744  0.159480  1.000000 -0.218836  0.160689  0.222576  0.160193   
sg    0.642156 -0.191096 -0.218836  1.000000 -0.469760 -0.296234 -0.374710   
al   -0.541993  0.122091  0.160689 -0.469760  1.000000  0.269305  0.379464   
su   -0.283416  0.220866  0.222576 -0.296234  0.269305  1.000000  0.717827   
bgr  -0.338673  0.244992  0.160193 -0.374710  0.379464  0.717827  1.000000   
bu   -0.307175  0.196985  0.188517 -0.314295  0.453528  0.168583  0.143322   
sc   -0.268683  0.132531  0.146222 -0.361473  0.399198  0.223244  0.114875   
sod   0.364251 -0.100046 -0.116422  0.412190 -0.459896 -0.131776 -0.267848   
pot  -0.092347  0.058377  0.075151 -0.072787  0.129038  0.219450  0.066966   
hemo  0.640298 -0.192928 -0.306540  0.602582 -0.634632 -0.224775

In [14]:
dt.replace('?', pd.NA, inplace=True)


In [16]:
dt.columns = dt.columns.str.strip()
dt = dt.applymap(lambda x: x.strip() if isinstance(x, str) else x)

<ipython-input-16-a317ca634abf>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dt = dt.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [18]:
# Convert specific columns to numeric (coerce errors to NaN)
cols_to_numeric = ['pcv', 'wc', 'rc']
for col in cols_to_numeric:
    dt[col] = pd.to_numeric(dt[col], errors='coerce')

In [19]:
replacements = {
    'rbc': {'normal': 0, 'abnormal': 1},
    'pc': {'normal': 0, 'abnormal': 1},
    'pcc': {'notpresent': 0, 'present': 1},
    'ba': {'notpresent': 0, 'present': 1},
    'htn': {'no': 0, 'yes': 1},
    'dm': {'no': 0, 'yes': 1, 'yes ' : 1, ' no': 0},
    'cad': {'no': 0, 'yes': 1},
    'appet': {'good': 0, 'poor': 1},
    'pe': {'no': 0, 'yes': 1},
    'ane': {'no': 0, 'yes': 1},
    'classification': {'ckd': 1, 'notckd': 0}
}

for col, mapping in replacements.items():
    if col in dt.columns:
        dt[col] = dt[col].map(mapping)

# Drop rows with any remaining nulls
dt_cleaned = dt.dropna()

# Show cleaned DataFrame info
print(dt_cleaned.info())
dt_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Index: 158 entries, 3 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              158 non-null    int64  
 1   age             158 non-null    float64
 2   bp              158 non-null    float64
 3   sg              158 non-null    float64
 4   al              158 non-null    float64
 5   su              158 non-null    float64
 6   rbc             158 non-null    float64
 7   pc              158 non-null    float64
 8   pcc             158 non-null    float64
 9   ba              158 non-null    float64
 10  bgr             158 non-null    float64
 11  bu              158 non-null    float64
 12  sc              158 non-null    float64
 13  sod             158 non-null    float64
 14  pot             158 non-null    float64
 15  hemo            158 non-null    float64
 16  pcv             158 non-null    float64
 17  wc              158 non-null    float64


,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
3,3,48.0,70.0,1.005,4.0,0.0,0.0,1.0,1.0,0.0,...,32.0,6700.0,3.9,1.0,0.0,0.0,1.0,1.0,1.0,1
9,9,53.0,90.0,1.020,2.0,0.0,1.0,1.0,1.0,0.0,...,29.0,12100.0,3.7,1.0,1.0,0.0,1.0,0.0,1.0,1
11,11,63.0,70.0,1.010,3.0,0.0,1.0,1.0,1.0,0.0,...,32.0,4500.0,3.8,1.0,1.0,0.0,1.0,1.0,0.0,1
14,14,68.0,80.0,1.010,3.0,2.0,0.0,1.0,1.0,1.0,...,16.0,11000.0,2.6,1.0,1.0,1.0,1.0,1.0,0.0,1
20,20,61.0,80.0,1.015,2.0,0.0,1.0,1.0,0.0,0.0,...,24.0,9200.0,3.2,1.0,1.0,1.0,1.0,1.0,1.0,1


In [20]:
dt.corr()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
id,1.000000,-0.185308,-0.245744,0.642156,-0.541993,-0.283416,-0.471301,-0.411358,-0.262235,-0.114213,...,0.630019,-0.198641,0.605072,-0.518609,-0.473578,-0.204724,-0.373787,-0.307475,-0.272523,-0.838528
age,-0.185308,1.000000,0.159480,-0.191096,0.122091,0.220866,0.079987,0.150867,0.157693,0.042160,...,-0.242119,0.118339,-0.268896,0.400254,0.369420,0.233560,0.161168,0.097621,0.053346,0.227268
bp,-0.245744,0.159480,1.000000,-0.218836,0.160689,0.222576,0.294129,0.189310,0.057891,0.111204,...,-0.326319,0.029753,-0.261936,0.271301,0.228946,0.085602,0.177914,0.054045,0.197345,0.294077
sg,0.642156,-0.191096,-0.218836,1.000000,-0.469760,-0.296234,-0.395774,-0.374407,-0.291618,-0.220656,...,0.603560,-0.236215,0.579476,-0.400085,-0.431046,-0.174722,-0.268498,-0.298618,-0.242699,-0.732163
al,-0.541993,0.122091,0.160689,-0.469760,1.000000,0.269305,0.437873,0.558535,0.401284,0.368373,...,-0.611891,0.231989,-0.566437,0.525756,0.413609,0.268324,0.357455,0.477274,0.321187,0.627090
su,-0.283416,0.220866,0.222576,-0.296234,0.269305,1.000000,0.164609,0.202815,0.151222,0.107047,...,-0.239189,0.184893,-0.237448,0.321933,0.523372,0.279979,0.087417,0.143085,0.075233,0.344070
rbc,-0.471301,0.079987,0.294129,-0.395774,0.437873,0.164609,1.000000,0.426395,0.110282,0.180657,...,-0.447670,-0.004201,-0.381538,0.306579,0.339051,0.193730,0.282708,0.302234,0.189951,0.555097
pc,-0.411358,0.150867,0.189310,-0.374407,0.558535,0.202815,0.426395,1.000000,0.514578,0.329655,...,-0.541718,0.134303,-0.548112,0.383096,0.301557,0.215118,0.316625,0.415723,0.329643,0.461812
pcc,-0.262235,0.157693,0.057891,-0.291618,0.401284,0.151222,0.110282,0.514578,1.000000,0.274491,...,-0.312942,0.189944,-0.350248,0.192673,0.162261,0.186738,0.187932,0.102485,0.174364,0.263226
ba,-0.114213,0.042160,0.111204,-0.220656,0.368373,0.107047,0.180657,0.329655,0.274491,1.000000,...,-0.208454,0.117600,-0.231433,0.086668,0.077761,0.161461,0.147872,0.133500,0.051001,0.185345


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Use the cleaned dataset (from earlier)
dt = dt_cleaned.copy()


In [24]:
# Features (all except target)
X = dt.drop(columns=['classification'])

# Target
y = dt['classification']


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [28]:
model = LogisticRegression(max_iter=1000)  # Increase if convergence warning
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [30]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00         9

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

Confusion Matrix:
 [[23  0]
 [ 0  9]]


In [37]:
import numpy as np

# Labels for the 25 input features (replace with your actual column names if different)
labels = [
    'age', 'blood pressure', 'specific gravity', 'albumin', 'sugar', 'red blood cells',
    'pus cell', 'pus cell clumps', 'bacteria', 'blood glucose random', 'blood urea',
    'serum creatinine', 'sodium', 'potassium', 'hemoglobin', 'packed cell volume',
    'white blood cell count', 'red blood cell count', 'hypertension', 'diabetes mellitus',
    'coronary artery disease', 'appetite', 'pedal edema', 'anemia', 'class'
]

# ✅ Default input for "Chronic Kidney Disease" (result should be 1)
default_ckd = [48, 80, 1.02, 1, 0, 1, 1, 1, 0, 121, 36, 1.2, 137, 4.5, 15, 44, 7800, 5.2, 1, 1, 0, 1, 1, 1, 1]

# ✅ Default input for "No Kidney Disease" (result should be 0)
default_normal = [25, 70, 1.015, 0, 0, 0, 0, 0, 0, 100, 18, 0.7, 138, 3.5, 15, 44, 8400, 5.0, 0, 0, 0, 1, 0, 0, 0]

# Choose which default to use
print("Choose test case:")
print("1: Chronic Kidney Disease")
print("2: No Kidney Disease")
choice = input("Enter 1 or 2: ")
defaults = default_ckd if choice == "1" else default_normal

# Take input from user with label and default
user_input = []
for i, label in enumerate(labels[:-1]):  # Exclude 'class'
    val = input(f"{label} (default={defaults[i]}): ") or str(defaults[i])
    user_input.append(float(val))

# Convert input to array and reshape
input_array = np.array(user_input).reshape(1, -1)

# Predict using your trained model
result = model.predict(input_array)

# Show result
if result[0] == 1:
    print("🩺 Prediction: Chronic Kidney Disease")
else:
    print("💚 Prediction: No Kidney Disease")


Choose test case:
1: Chronic Kidney Disease
2: No Kidney Disease
Enter 1 or 2: 1
age (default=48): 
blood pressure (default=80): 
specific gravity (default=1.02): 
albumin (default=1): 
sugar (default=0): 
red blood cells (default=1): 
pus cell (default=1): 
pus cell clumps (default=1): 
bacteria (default=0): 
blood glucose random (default=121): 
blood urea (default=36): 
serum creatinine (default=1.2): 
sodium (default=137): 
potassium (default=4.5): 
hemoglobin (default=15): 
packed cell volume (default=44): 
white blood cell count (default=7800): 
red blood cell count (default=5.2): 
hypertension (default=1): 
diabetes mellitus (default=1): 
coronary artery disease (default=0): 
appetite (default=1): 
pedal edema (default=1): 
anemia (default=1): 


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


ValueError: X has 24 features, but LogisticRegression is expecting 25 features as input.

In [39]:








import numpy as np
import pandas as pd # Import pandas for accessing column names
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming dt_cleaned is already loaded and preprocessed as before
# If not, ensure the previous steps to create dt_cleaned are run

# Use the cleaned dataset
dt = dt_cleaned.copy()

# Features (all except target)
X = dt.drop(columns=['classification'])

# Target
y = dt['classification']

# Split data (necessary to have X_train for training the model)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Default input for "Chronic Kidney Disease" (result should be 1)
# These defaults should have 25 values corresponding to the 25 features
default_ckd_values = [48, 80, 1.02, 1, 0, 1, 1, 1, 0, 121, 36, 1.2, 137, 4.5, 15, 44, 7800, 5.2, 1, 1, 0, 1, 1, 1, 1] # Note: These 25 values correspond to the 25 features + target in the original data structure. We only need the first 25 for prediction.

# Default input for "No Kidney Disease" (result should be 0)
default_normal_values = [25, 70, 1.015, 0, 0, 0, 0, 0, 0, 100, 18, 0.7, 138, 3.5, 15, 44, 8400, 5.0, 0, 0, 0, 1, 0, 0, 0] # Note: These 25 values correspond to the 25 features + target in the original data structure. We only need the first 25 for prediction.


# Use the actual column names from the training data (X) as labels
feature_labels = X.columns.tolist()

# Choose which default to use
print("Choose test case:")
print("1: Chronic Kidney Disease")
print("2: No Kidney Disease")
choice = input("Enter 1 or 2: ")

# Select the first 25 values from the default lists, corresponding to the features
defaults = default_ckd_values[:25] if choice == "1" else default_normal_values[:25]


# Take input from user with label and default
user_input = []
# Iterate using the actual feature labels
for i, label in enumerate(feature_labels):
    val = input(f"{label} (default={defaults[i]}): ") or str(defaults[i])
    user_input.append(float(val))

# Convert input to array and reshape
# This array will now have 25 features, matching the model's expectation
input_array = np.array(user_input).reshape(1, -1)
result_proba = model.predict_proba(input_array)
print("Probability of CKD:", result_proba[0][1])
print("Probability of No CKD:", result_proba[0][0])


# Show result
if result[0] == 1:
    print("🩺 Prediction: Chronic Kidney Disease")
else:
    print("💚 Prediction: No Kidney Disease")
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test)))














Choose test case:
1: Chronic Kidney Disease
2: No Kidney Disease
Enter 1 or 2: 1
id (default=48): 
age (default=80): 
bp (default=1.02): 
sg (default=1): 
al (default=0): 
su (default=1): 
rbc (default=1): 
pc (default=1): 
pcc (default=0): 
ba (default=121): 
bgr (default=36): 
bu (default=1.2): 
sc (default=137): 
sod (default=4.5): 
pot (default=15): 
hemo (default=44): 
pcv (default=7800): 
wc (default=5.2): 
rc (default=1): 
htn (default=1): 
dm (default=0): 
cad (default=1): 
appet (default=1): 
pe (default=1): 
ane (default=1): 
Probability of CKD: 2.0012390423344347e-234
Probability of No CKD: 1.0
💚 Prediction: No Kidney Disease
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00         9

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32



/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [40]:
# Show one CKD-positive row
dt[dt['classification'] == 1].iloc[0]


,3
id,3.000
age,48.000
bp,70.000
sg,1.005
al,4.000
su,0.000
rbc,0.000
pc,1.000
pcc,1.000
ba,0.000


In [41]:
default_inputs = {
    "id": 48,
    "age": 70,
    "bp": 80,
    "sg": 1.015,
    "al": 3,
    "su": 0,
    "rbc": 0,
    "pc": 0,
    "pcc": 1,
    "ba": 1,
    "bgr": 423,
    "bu": 53,
    "sc": 1.8,
    "sod": 137,
    "pot": 4.5,
    "hemo": 11.3,
    "pcv": 33,
    "wc": 6700,
    "rc": 3.9,
    "htn": 1,
    "dm": 1,
    "cad": 0,
    "appet": 0,
    "pe": 0,
    "ane": 1
}


In [42]:
import numpy as np

# Let user override or accept default inputs
user_input = []
print("Enter patient details (press Enter to accept default values):")
for key, default in default_inputs.items():
    val = input(f"{key} (default={default}): ")
    if val.strip() == "":
        user_input.append(default)
    else:
        try:
            user_input.append(float(val))
        except ValueError:
            print(f"Invalid input for {key}, using default {default}")
            user_input.append(default)

# Convert to NumPy array and reshape
input_array = np.array(user_input).reshape(1, -1)

# Predict and get probabilities
prob = model.predict_proba(input_array)
prediction = model.predict(input_array)

# Display probabilities
print(f"\nProbability of CKD: {prob[0][1]}")
print(f"Probability of No CKD: {prob[0][0]}")

# Show final prediction
if prediction[0] == 1:
    print("💀 Prediction: Chronic Kidney Disease")
else:
    print("💚 Prediction: No Kidney Disease")


Enter patient details (press Enter to accept default values):
id (default=48): 
age (default=70): 
bp (default=80): 
sg (default=1.015): 
al (default=3): 
su (default=0): 
rbc (default=0): 
pc (default=0): 
pcc (default=1): 
ba (default=1): 
bgr (default=423): 
bu (default=53): 
sc (default=1.8): 
sod (default=137): 
pot (default=4.5): 
hemo (default=11.3): 
pcv (default=33): 
wc (default=6700): 
rc (default=3.9): 
htn (default=1): 
dm (default=1): 
cad (default=0): 
appet (default=0): 
pe (default=0): 
ane (default=1): 

Probability of CKD: 1.0
Probability of No CKD: 0.0
💀 Prediction: Chronic Kidney Disease


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
